## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Avarua,CK,77.00,clear sky,-21.21,-159.78,Paradise Inn
1,2,Butaritari,KI,83.19,overcast clouds,3.07,172.79,Isles Sunset Lodge
2,11,Kapaa,US,86.00,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
3,13,Nouadhibou,MR,73.40,broken clouds,20.93,-17.03,El Medina
4,14,Auki,SB,83.95,moderate rain,-8.77,160.70,Auki Motel
5,16,Sola,VU,86.00,broken clouds,-13.88,167.55,Leumerus Bungalows
6,25,Mareeba,AU,82.40,few clouds,-17.00,145.43,Trinity Plains Tourist Park
7,26,Eydhafushi,MV,83.71,clear sky,5.10,73.07,Soneva Fushi
8,27,Dafeng,CN,84.13,clear sky,23.45,108.59,Shenglong Hotel
9,33,Kalmunai,LK,78.55,clear sky,7.42,81.82,VS Villa


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
Current_Description = vacation_df["Current Description"]

In [4]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start= vacation_df.loc[(vacation_df["Hotel Name"] == "The Clift Royal Sonesta Hotel") & (vacation_df["City"] == "San Francisco")]
vacation_end = vacation_df.loc[(vacation_df["Hotel Name"] == "The Clift Royal Sonesta Hotel") & (vacation_df["City"] == "San Francisco")]
vacation_stop1 = vacation_df.loc[(vacation_df["Hotel Name"] == "The Westin San Jose") & (vacation_df["City"] == "San Jose")]
vacation_stop2 = vacation_df.loc[(vacation_df["Hotel Name"] == "Lovers Point Inn") & (vacation_df["City"] == "Pacific Grove")] 
vacation_stop3 = vacation_df.loc[(vacation_df["Hotel Name"] == "Kimpton Canary Hotel") & (vacation_df["City"] == "Santa Barbara")]
vacation_start.dtypes                                

City_ID                  int64
City                    object
Country                 object
Max Temp               float64
Current Description     object
Lat                    float64
Lng                    float64
Hotel Name              object
dtype: object

In [6]:
import numpy as np

In [7]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.

start = vacation_start[['Lat', 'Lng']]
start_tuples = [tuple(x) for x in start.to_numpy()]
end = vacation_end[['Lat', 'Lng']]
end_tuples = [tuple(x) for x in end.to_numpy()]

stop1 = vacation_stop1[['Lat', 'Lng']]
stop1_tuples = [tuple(x) for x in stop1.to_numpy()]

stop2 = vacation_stop2[['Lat', 'Lng']]
stop2_tuples = [tuple(x) for x in stop2.to_numpy()]

stop3 = vacation_stop3[['Lat', 'Lng']]
stop3_tuples = [tuple(x) for x in stop3.to_numpy()]

In [8]:
len(start_tuples[0])

2

In [9]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs, and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

# start = (37.77, -122.42)
#end = (37.77, -122.42)
# waypoints = [(37.34, -121.89), (36.62, -121.92), (34.42, -119.7)] , 
waypoints = [stop1_tuples[0], stop2_tuples[0], stop3_tuples[0]]
fig = gmaps.figure()
directions = gmaps.directions_layer(start_tuples[0], end_tuples[0], waypoints = waypoints, travel_mode="DRIVING")
fig.add_layer(directions)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_stop3, vacation_stop2, vacation_stop1, vacation_start],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,372,Santa Barbara,US,84.0,haze,34.42,-119.70,Kimpton Canary Hotel
1,371,Pacific Grove,US,75.0,clear sky,36.62,-121.92,Lovers Point Inn
2,273,San Jose,US,84.2,smoke,37.34,-121.89,The Westin San Jose
3,677,San Francisco,US,77.0,smoke,37.77,-122.42,The Clift Royal Sonesta Hotel


In [11]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]
max_temp = itinerary_df["Max Temp"]
Current_Description = itinerary_df["Current Description"]

In [12]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))